In [1]:
import pandas as pd #data processing

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

test=test.fillna(' ') #fills null or empty cells for test 
train=train.fillna(' ') #fills null or empty cells for train 

test['total']=test['title']+' '+test['author']+test['text'] #combines the three columns to one column
train['total']=train['title']+' '+train['author']+train['text'] #combines the three columns to one column

In [2]:
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from nltk import word_tokenize

lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english') #Stores english stopwords to a variable

for index,row in train.iterrows():
    filter_sentence = ''
    
    sentence = row['total']
    sentence = re.sub(r'[^\w\s]','',sentence) #cleaning - removes punctuation
    
    words = word_tokenize(sentence) #tokenization
    
    words = [w for w in words if not w in stop_words]  #stopwords removal
    
    for word in words:
        filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower() #lemmatization and combining of words
                                                                                            # to sentence
    train.loc[index,'total'] = filter_sentence #replace total value with cleaned sentence

In [3]:
X_train = train['total']
Y_train = train['label']

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vectorizer = CountVectorizer() 
count_vectorizer.fit_transform(X_train) #learns the counts how many times a word is used
freq_term_matrix = count_vectorizer.transform(X_train) #counts the times a word is used and puts it in a matrix
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix) #learns the original value of the words
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix) #converts the value in the array using tfidf 


In [5]:
from sklearn.model_selection import train_test_split

test_counts = count_vectorizer.transform(test['total'].values) #counts the times a word is used and puts it in a matrix
test_tfidf = tfidf.transform(test_counts)  #converts the value in the array using tfidf 

X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, Y_train, random_state=0) #randomizes the place ment of the values

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

logreg = LogisticRegression(C=1e5) 
logreg.fit(X_train, y_train) 
pred = logreg.predict(X_test) 
print('Accuracy of Lasso classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Lasso classifier on test set: {:.2f}'
     .format(logreg.score(X_test, y_test)))
print(pred)
cm = confusion_matrix(y_test, pred) 
cm

C:\Users\zeldr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Lasso classifier on training set: 1.00
Accuracy of Lasso classifier on test set: 0.98
[0 1 1 ... 0 1 0]


array([[2495,   69],
       [  44, 2592]], dtype=int64)

In [7]:
NB = MultinomialNB()
NB.fit(X_train, y_train)
pred = NB.predict(X_test)
print('Accuracy of NB  classifier on training set: {:.2f}'
     .format(NB.score(X_train, y_train)))
print('Accuracy of NB classifier on test set: {:.2f}'
     .format(NB.score(X_test, y_test)))
cm = confusion_matrix(y_test, pred)
cm

Accuracy of NB  classifier on training set: 0.88
Accuracy of NB classifier on test set: 0.83


array([[2558,    6],
       [ 853, 1783]], dtype=int64)

In [8]:
X_train = train['total']
Y_train = train['label']

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm='l2')),
    ('clf', LogisticRegression(C=1e5, solver='lbfgs')),
])
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1

In [10]:
import cv2
import pytesseract as pt

img = cv2.imread('C:\\Users\\zeldr\\Desktop\\Thesis\\image_uploads\\real.png') #Thesis Dataset\\News\\new_real\\Fake\\fake9.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
adaptive_threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 15)
text = pt.image_to_string(adaptive_threshold, config = "--psm 1")
stop_words = stopwords.words('english')
text = re.sub(r'[^\w\s]', '', text)
print(pipeline.predict([text]))

himg, wimg, _ = img.shape
boxes = pt.image_to_data(img)
for x,b in enumerate(boxes.splitlines()):
    if x!=0:
        b = b.split()
        if len(b)==12:
            x,y,w,h = int(b[6]), int(b[7]), int(b[8]), int(b[9])
            cv2.rectangle(img,(x,y),(w+x,h+y),(0,0,255),1)
            #cv2.putText(img, b[11], (x,y), cv2.FONT_HERSHEY_COMPLEX,1,(50,50,255),2)
#cv2.imshow('Result', img)
print(text)
#cv2.waitKey(0)

[1]
US military to accept transgender recruits on Monday Pentagon

Transgender people will be allowed for the first time to enlist in the
US military starting on Monday as ordered by federal courts the
Pentagon said on Friday after President Donald Trumps administration
decided not to appeal rulings that blocked his transgender ban Two
federal appeals courts one in Washington and one in Virginia last
week rejected the administrations request to put on hold orders by
lower court judges requiring the military to begin accepting
transgender recruits on Jan 1 A Justice Department official said the
administration will not challenge those rulings The Department of
Defense has announced that it will be releasing an independent study
of these issues in the coming weeks So rather than litigate this interim
appeal before that occurs the administration has decided to wait for
DODs study and will continue to defend the president

  

 



In [11]:
X_train = train['total']
Y_train = train['label']

count_vectorizer = CountVectorizer() 
count_vectorizer.fit_transform(X_train) #learns the counts how many times a word is used
freq_term_matrix = count_vectorizer.transform(X_train) #counts the times a word is used and puts it in a matrix
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix) #learns the original value of the words
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix) #converts the value in the array using tfidf 

test_counts = count_vectorizer.transform(test['total'].values) #counts the times a word is used and puts it in a matrix
test_tfidf = tfidf.transform(test_counts)  #converts the value in the array using tfidf 

X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, Y_train, random_state=0) #randomizes the place ment of the values



In [12]:
filter_sentence = ''
    
sentence = text
sentence = re.sub(r'[^\w\s]','',sentence) #cleaning - removes punctuation

words = word_tokenize(sentence) #tokenization

words = [w for w in words if not w in stop_words]  #stopwords removal

for word in words:
    filter_sentence = filter_sentence + ' ' + str(lemmatizer.lemmatize(word)).lower() #lemmatization and combining of words


In [13]:
logreg = LogisticRegression(C=1e5) 
logreg.fit(X_train, y_train) 
new_text1 = count_vectorizer.transform([filter_sentence])
new_text2 = tfidf.transform(new_text1)
pred = logreg.predict(new_text2)
print(pred)

C:\Users\zeldr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0]


In [14]:
print(new_text1)

  (0, 8551)	1
  (0, 8563)	1
  (0, 9653)	4
  (0, 12511)	1
  (0, 14743)	1
  (0, 16180)	3
  (0, 21240)	1
  (0, 23266)	1
  (0, 26457)	1
  (0, 34729)	1
  (0, 39395)	1
  (0, 41549)	1
  (0, 43156)	3
  (0, 47357)	2
  (0, 47762)	1
  (0, 47784)	1
  (0, 48926)	2
  (0, 53109)	1
  (0, 58957)	1
  (0, 64475)	2
  (0, 65869)	1
  (0, 68702)	1
  (0, 81977)	1
  (0, 87695)	1
  (0, 89504)	1
  :	:
  (0, 131724)	2
  (0, 131776)	1
  (0, 138206)	2
  (0, 141290)	1
  (0, 143570)	1
  (0, 144868)	2
  (0, 146074)	1
  (0, 146220)	1
  (0, 147164)	1
  (0, 147181)	1
  (0, 151218)	2
  (0, 152456)	2
  (0, 162419)	1
  (0, 165715)	1
  (0, 167668)	2
  (0, 173473)	1
  (0, 174968)	1
  (0, 176936)	4
  (0, 178197)	1
  (0, 179219)	1
  (0, 182557)	2
  (0, 185554)	1
  (0, 186917)	1
  (0, 187593)	1
  (0, 188253)	2


In [15]:
print(new_text2)

  (0, 188253)	0.0938618588677793
  (0, 187593)	0.059313487638442124
  (0, 186917)	0.08753505464165773
  (0, 185554)	0.09171348071103368
  (0, 182557)	0.10716705736658097
  (0, 179219)	0.04072487030076729
  (0, 178197)	0.05557673484188807
  (0, 176936)	0.4688627306490734
  (0, 174968)	0.03602750411447628
  (0, 173473)	0.02494947580785904
  (0, 167668)	0.14734927977561627
  (0, 165715)	0.08526057784506738
  (0, 162419)	0.05608855447799981
  (0, 152456)	0.0629317026822046
  (0, 151218)	0.17934636432049342
  (0, 147181)	0.11002648297769207
  (0, 147164)	0.08172780279339911
  (0, 146220)	0.1118047942002317
  (0, 146074)	0.09406274508354623
  (0, 144868)	0.23669675213068359
  (0, 143570)	0.06652245163083317
  (0, 141290)	0.05329241271007028
  (0, 138206)	0.08175122502986702
  (0, 131776)	0.03474331300684489
  (0, 131724)	0.200417200842842
  :	:
  (0, 89504)	0.12738722337117264
  (0, 87695)	0.07949815630065822
  (0, 81977)	0.06898969671798269
  (0, 68702)	0.06633551722673221
  (0, 65869)	0.04

In [16]:
feature_names = count_vectorizer.get_feature_names()
sorted(zip(logreg.coef_[0], feature_names), reverse=True)[:20]

[(54.64290146380723, 'us'),
 (40.65829342566536, 'october'),
 (39.63164351365938, '2016'),
 (34.911539500895096, 'november'),
 (27.3626048124683, 'presidentelect'),
 (23.203034376156797, 'socalled'),
 (22.435584131828296, 'oct'),
 (22.411680093393763, 'share'),
 (22.201009682254846, 'of'),
 (20.47769936679514, 'however'),
 (19.926467764603647, 'comment'),
 (19.898375810690368, 'snip'),
 (19.798713962933668, 'giuliani'),
 (19.634271040127018, 'altright'),
 (19.551621076506763, 'advertisement'),
 (19.444796988218975, 'source'),
 (19.153069829799875, 'tags'),
 (18.23018920720736, 'this'),
 (17.936784994598067, 'via'),
 (17.59413201614912, 'republican')]

In [17]:
real_words=sorted(zip(logreg.coef_[0], feature_names))[:100]

In [18]:
df = pd.DataFrame(real_words, columns = ['Weight', 'Word']) 
df.to_csv('Real Words.csv', index=False)

In [20]:
import eli5
eli5.show_weights(logreg, feature_names=feature_names)

In [21]:
eli5.show_prediction(logreg, new_text2, feature_names=feature_names)

import joblib

filename = 'NLP.sav'

joblib.dump(pipeline, filename)

real_words = ''
fake_words = ''
stopwords = set(stop_words) 
  
# iterate through the csv file 
for val in train[train['label']==0].total: 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    real_words += " ".join(tokens)+" "

for val in train[train['label']==1].total: 
      
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    fake_words += " ".join(tokens)+" "

from wordcloud import WordCloud

import matplotlib.pyplot as plt 
%matplotlib inline

wordcloud_fake = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(fake_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud_fake) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

wordcloud_real = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(real_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud_real) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 